In [0]:
!pip install mlflow --quiet
dbutils.library.restartPython()

In [0]:
import mlflow
email = ''
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment(f"/Users/{email}/5-prediccion-infarto")

In [0]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

# Predicción de Enfermedades del Corazón

En este ejemplo, trabajaremos con el famoso conjunto de datos "UCI Heart Disease". Este conjunto de datos contiene un conjunto de atributos relacionados con pacientes potencialmente afectados por una enfermedad cardiovascular (ECV). Las ECV son una de las principales causas de mortalidad, pero se estima que hasta un 90% de las ECV podrían ser prevenibles. Un diagnóstico temprano podría ser esencial en la mayoría de los casos y la inteligencia artificial puede lograr este objetivo.


## Clasificación Binaria en Aprendizaje Automático

### Entrenamiento de un Clasificador Binario
1. **Preparación de Datos**: Divide el conjunto de datos en entrenamiento y prueba.
2. **Selección del Modelo**: Elige un modelo de clasificación binaria (ej. regresión logística).
3. **Entrenamiento del Modelo**: Entrena el modelo usando una biblioteca como sklearn.
4. **Ajuste de Hiperparámetros**: Mejora el rendimiento ajustando los hiperparámetros.

### Evaluación del Clasificador Binario
1. **Matriz de Confusión**: Muestra predicciones correctas e incorrectas divididas en dos clases.
2. **Precisión (Precision)**: Proporción de identificaciones positivas correctas.
   - `Precisión = VP / (VP + FP)`
3. **Recall (Sensibilidad)**: Proporción de positivos reales identificados correctamente.
   - `Recall = VP / (VP + FN)`
4. **Puntuación F1 (F1 Score)**: Promedio ponderado de precisión y recall.
   - `F1 = 2 * (Precisión * Recall) / (Precisión + Recall)`
5. **Curva ROC y AUC**: Rendimiento del clasificador y medida agregada de rendimiento.
6. **Validación Cruzada**: Evalúa la robustez del modelo.

In [0]:
# leemos los datos
data = pd.read_csv('../data/raw/heart.csv')

In [0]:
# le damos una vueltecilla a ver que pintan tienen...
data.head()

1. **age**: edad en años.
2. **sex**: sexo (1 = masculino; 0 = femenino).
3. **cp**: tipo de dolor de pecho.
   - Valor 1: angina típica.
   - Valor 2: angina atípica.
   - Valor 3: dolor no anginoso.
   - Valor 4: asintomático.
4. **trestbps**: presión arterial en reposo (en mm Hg al ingresar al hospital).
5. **chol**: colesterol sérico en mg/dl.
6. **fbs**: azúcar en sangre en ayunas > 120 mg/dl (1 = verdadero; 0 = falso).
7. **restecg**: resultados electrocardiográficos en reposo.
   - Valor 0: normal.
   - Valor 1: con anormalidad de onda ST-T (inversiones de onda T y/o elevación o depresión del ST de > 0.05 mV).
   - Valor 2: muestra probable o definitiva hipertrofia ventricular izquierda según los criterios de Estes.
8. **thalach**: frecuencia cardíaca máxima alcanzada.
9. **exang**: angina inducida por el ejercicio (1 = sí; 0 = no).
10. **oldpeak**: depresión del segmento ST inducida por el ejercicio en relación al reposo.
11. **slope**: la pendiente del segmento ST durante el pico del ejercicio.
    - Valor 1: ascendente.
    - Valor 2: plano.
    - Valor 3: descendente.
12. **ca**: número de vasos principales (0-3) coloreados por fluoroscopia.
13. **thal**: 3 = normal; 6 = defecto fijo; 7 = defecto reversible.
14. **target**: variable objetivo, se refiere a la presencia de enfermedad cardíaca en el paciente.

In [0]:
data.info()

In [0]:
data.describe()

La edad media es de 54 años, los adultos tienen más probabilidades que los jóvenes de sufrir enfermedades cardiovasculares.

In [0]:
# Vamos a echarle un ojo al target
data.target.value_counts()

El target está balanceado y el dataset un poco sucio :P

In [0]:
# test and train split
train_set, test_set = train_test_split(data, test_size=0.2, random_state=5)

In [0]:
# Vamos a ver distribuciones
train_set.hist(bins=50, figsize=(20, 15))

The feature have different scale, it's a good idea to perform standard scaling

In [0]:
# Creeamos un pipeline para hacer un one hot encoding de las variables categoricas y además, hacemos un median target encoding para los valores nulos.
# finalmente estandarizamos las escalas de las columnas
cat_attr = ["sex", "cp", "fbs", "restecg", "exang", "slope"]
num_attr = ["age", "trestbps", "chol", "thalach", "oldpeak", "ca", "thal"]

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("std_scaler", StandardScaler())
])

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attr),
    ("cat", OneHotEncoder(), cat_attr)
])

In [0]:
x_train = train_set.drop("target", axis=1)
y_train = train_set.target

In [0]:
x_train_pr = full_pipeline.fit_transform(x_train)

# Entrenamiento del modelo y evaluacion

Vamos a probar con un simple clasificador binario como base, en este caso un Clasificador estocástico por descenso de gradiente (SGD)

In [0]:
sgd_clf = SGDClassifier(random_state=42)

Lo normal es hacer validación cruzada asique lo hacemos

In [0]:
scores = cross_val_score(sgd_clf, x_train_pr, y_train, cv=3, scoring="accuracy")
scores.mean()

Vamos a pintar una matriz de confusión

In [0]:
# Es como hacer validacion cruzada pero retornando las predicciones
preds = cross_val_predict(sgd_clf, x_train_pr, y_train, cv=3)

In [0]:
# Ahora podemos pintar la matriz de confusion
cm = confusion_matrix(y_train, preds)
ConfusionMatrixDisplay(cm).plot()

Vamos a ver precision, recall y f1 socre

In [0]:
print("Precision: ", precision_score(y_train, preds))
print("Recall: ", recall_score(y_train, preds))
print("F1 score: ", f1_score(y_train, preds))

In [0]:
# Otra metrica top es el roc auc score
print("roc auc score:", roc_auc_score(y_train, preds))

Creo que ya es hora que usemos un modelo mas potente como el RandomForestClassifier

In [0]:
rf_clf = RandomForestClassifier(random_state=42)
rf_preds = cross_val_predict(rf_clf, x_train_pr, y_train, cv=3)

In [0]:
cm = confusion_matrix(y_train, rf_preds)
ConfusionMatrixDisplay(cm).plot()

In [0]:
# Pintamos metricas
print("Precision: ", precision_score(y_train, rf_preds))
print("Recall: ", recall_score(y_train, rf_preds))
print("F1 score: ", f1_score(y_train, rf_preds))
print("roc auc score:", roc_auc_score(y_train, rf_preds))

Un poco mejor que el modelo base, vamos a usar el dataset entero para entrenar el RF Classifier y vemos que tal

In [0]:
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(x_train_pr, y_train)

In [0]:
x_test = test_set.drop("target", axis=1)
y_test = test_set.target

In [0]:
x_test_pr = full_pipeline.transform(x_test)
final_preds = forest_clf.predict(x_test_pr)

In [0]:
# Print the final metrics
print("Precision: ", precision_score(y_test, final_preds))
print("Recall: ", recall_score(y_test, final_preds))
print("F1 score: ", f1_score(y_test, final_preds))
print("roc auc score:", roc_auc_score(y_test, final_preds))

Este fue un ejemplo sencillo de cómo evaluar un clasificador, ¡sin embargo, el resultado es bueno! La validación cruzada es un buen método para evaluar modelos, pero como dividimos nuestro conjunto de datos en 3 partes, el modelo tuvo pocos datos para lograr buenos resultados. ¡Con el conjunto de datos completo alcanzamos un buen resultado!